In [ ]:
from google.colab import drive
drive.mount("/content/gdrive/")

Mounted at /content/gdrive/


In [ ]:
!pip install optuna

In [ ]:
import numpy as np
import pandas as pd

from plotly.offline import init_notebook_mode
from plotly.subplots import make_subplots
import plotly.graph_objects as go

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import optuna

from torch.utils.data import Dataset, DataLoader

from sklearn.preprocessing import StandardScaler

from itertools import chain

## Util functions

In [ ]:
def plot_series(df, height=1200):
    N = len(df)
    fig = make_subplots(rows=N, cols=1, subplot_titles=df.index)

    for i in range(N):
        fig.add_trace(
            go.Scatter(x=df.columns, y=df.values[i], line=dict(color="black"), showlegend=False),
            row=i+1, col=1
        )

    fig.update_layout(height=height)
    fig.show()
def train_test_split(df, horizon):
    return df[df.columns[:-horizon]], df[df.columns[-horizon:]]
class TimeSeriesDataset(Dataset):
    def __init__(self, df, horizon=28, context_length=None):
        self.segments = df.index
        self.columns = df.columns
        self.values = df.values
        self.horizon = horizon
        self.context = context_length
        if self.context is None:
            self.context = 3 * self.horizon

        self.samples_in_series = (len(self.columns) - (horizon + self.context) + 1)
        self.length = self.samples_in_series * len(self.segments)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        row = idx // self.samples_in_series
        col = idx % self.samples_in_series
        return torch.from_numpy(self.values[row, col:col+self.context]), torch.from_numpy(self.values[row, col+self.context:col+self.context+self.horizon])

In [ ]:
def train_test_split(df, horizon):
    return df[df.columns[:-horizon]], df[df.columns[-horizon:]]

In [ ]:
class TimeSeriesDataset(Dataset):
    def __init__(self, df, horizon=28, context_length=None):
        self.segments = df.index
        self.columns = df.columns
        self.values = df.values
        self.horizon = horizon
        self.context = context_length
        if self.context is None:
            self.context = 3 * self.horizon

        self.samples_in_series = (len(self.columns) - (horizon + self.context) + 1)
        self.length = self.samples_in_series * len(self.segments)

    def __len__(self):
        return self.length

    def __getitem__(self, idx):
        row = idx // self.samples_in_series
        col = idx % self.samples_in_series
        return torch.from_numpy(self.values[row, col:col+self.context]), torch.from_numpy(self.values[row, col+self.context:col+self.context+self.horizon])

In [ ]:
class Forecaster(nn.Module):
    def smooth_series(self, X):
        output = []
        for degree in self.smooth_degrees:
            smoothed = []
            for i in range(self.context - degree + 1):
                smoothed.append(X[:, i : i+degree].mean(dim=1).unsqueeze(0))
            smoothed = torch.cat(smoothed, dim=0).T
            output.append(smoothed)
        return output

    def downsample_series(self, X):
        output = []
        for rate in self.ds_rates:
            output.append(X[:, ::rate])
        return output

    def __init__(
        self, 
        context,
        horizon,
        smooth_degrees=[], 
        ds_rates=[], 
        kernels=64,
        pooling="max", 
        kernel_size=5,
        pooling_factor=2
    ):
        super().__init__()
        self.context = context
        self.horizon = horizon
        self.smooth_degrees = smooth_degrees
        self.ds_rates = ds_rates
        self.kernels = kernels
        self.pooling_factor = pooling_factor
        self.series_num = 1 + len(smooth_degrees) + len(ds_rates)

        self.convs_1 = nn.ModuleList([
            nn.Conv1d(1, kernels, kernel_size=kernel_size) for _ in range(self.series_num)
        ])
        self.pools = None
        if pooling == "max":
            self.pools = nn.ModuleList([
                nn.AdaptiveMaxPool1d(pooling_factor) for _ in range(self.series_num)
            ])
        elif pooling == "avg":
            self.pools = nn.ModuleList([
                nn.AdaptiveAvgPool1d(pooling_factor) for _ in range(self.series_num)
            ])

        # scenario 1: (N, C * K, p)
        self.conv_2 = nn.Conv1d(
            in_channels=kernels * self.series_num,
            out_channels=kernels,
            kernel_size=pooling_factor
        )

        self.linear = nn.Linear(
            in_features=kernels,
            out_features=horizon
        )

    def forward(self, X):
        smoothed = self.smooth_series(X)
        downsample = self.downsample_series(X)

        series = [X.unsqueeze(dim=1)]
        for X_i in chain(smoothed, downsample):
            series.append(X_i.unsqueeze(dim=1))
        # series = list of (N, 1, L_i)
        output = []
        for conv, pool, X_i in zip(self.convs_1, self.pools, series):
            output.append(pool(conv(X_i)))
        
        # scenario 1: (N, C * K, p)
        output = torch.cat(output, dim=1)
        output = self.conv_2(output).squeeze()
        output = self.linear(output)
        return output

    def count_params(self):
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return params



In [ ]:
class LSTMForecaster(nn.Module):
    def __init__(self, horizon, device, hidden=64):
        super().__init__()
        self.hidden = hidden
        self.horizon = horizon
        self.device = device

        self.encoder = nn.LSTM(
            input_size=1,
            hidden_size=hidden,
            batch_first=True
        )

        self.decoder = nn.LSTM(
            input_size=1,
            hidden_size=hidden,
            batch_first=True
        )

        self.linear = nn.Linear(
            in_features=self.hidden,
            out_features=1
        )

    def forward(self, X):
        h_0, c_0 = self.initial_states(X.size()[0])
        _, (h_n, c_n) = self.encoder(X.unsqueeze(2), (h_0, c_0))
        
        inputs = X[:, -1].unsqueeze(1).unsqueeze(2) # taking last value

        outputs = []
        for i in range(self.horizon):
            inputs, (h_n, c_n)  = self.decoder(inputs, (h_n, c_n))
            outputs.append(self.linear(inputs.squeeze()))
            inputs = outputs[i].unsqueeze(1)

        return torch.cat(outputs, 1)


    def initial_states(self, N):
        h = torch.rand(1, N, self.hidden).to(self.device)
        c = torch.rand(1, N, self.hidden).to(self.device)
        return h, c

    def count_params(self):
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return params

In [ ]:
class ConvLSTM(nn.Module):
    def smooth_series(self, X):
        output = []
        for degree in self.smooth_degrees:
            smoothed = []
            for i in range(self.context - degree + 1):
                smoothed.append(X[:, i : i+degree].mean(dim=1).unsqueeze(0))
            smoothed = torch.cat(smoothed, dim=0).T
            output.append(smoothed)
        return output

    def downsample_series(self, X):
        output = []
        for rate in self.ds_rates:
            output.append(X[:, ::rate])
        return output

    def __init__(
        self, 
        context,
        horizon,
        smooth_degrees=[], 
        ds_rates=[], 
        kernels=64,
        pooling="max", 
        kernel_size=5,
        pooling_factor=2,
        lstm_hidden=64
    ):
        super().__init__()
        self.context = context
        self.horizon = horizon
        self.smooth_degrees = smooth_degrees
        self.ds_rates = ds_rates
        self.kernels = kernels
        self.pooling_factor = pooling_factor
        self.series_num = 1 + len(smooth_degrees) + len(ds_rates)

        self.convs_1 = nn.ModuleList([
            nn.Conv1d(1, kernels, kernel_size=kernel_size) for _ in range(self.series_num)
        ])
        self.pools = None
        if pooling == "max":
            self.pools = nn.ModuleList([
                nn.AdaptiveMaxPool1d(pooling_factor) for _ in range(self.series_num)
            ])
        elif pooling == "avg":
            self.pools = nn.ModuleList([
                nn.AdaptiveAvgPool1d(pooling_factor) for _ in range(self.series_num)
            ])

        # scenario 1: (N, C * K, p)
        self.conv_2 = nn.Conv1d(
            in_channels=kernels * self.series_num,
            out_channels=kernels,
            kernel_size=pooling_factor
        )

        self.decoder = nn.LSTM(
            input_size=1,
            hidden_size=lstm_hidden,
            batch_first=True
        )

        self.hidden_linear = nn.Linear(
            in_features=kernels,
            out_features=lstm_hidden
        )

        self.cell_linear = nn.Linear(
            in_features=kernels,
            out_features=lstm_hidden
        )

        self.linear = nn.Linear(
            in_features=lstm_hidden,
            out_features=1
        )

    def forward(self, X):
        smoothed = self.smooth_series(X)
        downsample = self.downsample_series(X)

        series = [X.unsqueeze(dim=1)]
        for X_i in chain(smoothed, downsample):
            series.append(X_i.unsqueeze(dim=1))
        # series = list of (N, 1, L_i)
        output = []
        for conv, pool, X_i in zip(self.convs_1, self.pools, series):
            output.append(pool(conv(X_i)))
        
        # scenario 1: (N, C * K, p)
        output = torch.cat(output, dim=1)
        output = self.conv_2(output).squeeze() # (N, kernels)

        h_n, c_n = self.hidden_linear(output).unsqueeze(0), self.cell_linear(output).unsqueeze(0) # (1, N, hidden)
        inputs = X[:, -1].unsqueeze(1).unsqueeze(2) # (N, L=1, H_in=1)

        outputs = []
        for i in range(self.horizon):
            inputs, (h_n, c_n)  = self.decoder(inputs, (h_n, c_n))
            outputs.append(self.linear(inputs.squeeze()))
            inputs = outputs[i].unsqueeze(1)

        return torch.cat(outputs, 1)

    def count_params(self):
        model_parameters = filter(lambda p: p.requires_grad, self.parameters())
        params = sum([np.prod(p.size()) for p in model_parameters])
        return params



In [ ]:
def train(forecaster, loss_function, optimizer, trainloader, EPOCHS, device, scheduler=None, disp=False, df_test=None, df_train=None, context_length=0, scaler=None):
    for epoch in range(EPOCHS):  # loop over the dataset multiple times

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            context, actual = data
            context = context.type(torch.FloatTensor).to(device)
            actual = actual.type(torch.FloatTensor).to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            output = forecaster(context)
            loss = loss_function(output, actual)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            
        if disp:
            df_metrics = metrics(df_test, forecast(df_train, df_test, forecaster, context_length, device), scaler)
            print("Epoch {}: Loss={}, test SMAPE={}".format(epoch, running_loss / len(trainloader), df_metrics.mean()["SMAPE"]))

        if scheduler is not None:
                scheduler.step(running_loss / len(trainloader))

    print('Finished Training')

In [ ]:
def forecast(train_df, test_df, model, context, device):
    to_forecast = torch.from_numpy(train_df.values[:, -context:]).type(torch.FloatTensor).to(device)
    forecast = model(to_forecast)
    forecast = pd.DataFrame(
        index=test_df.index,
        columns=test_df.columns,
        data=forecast.cpu().detach().numpy()
    )
    return forecast

In [ ]:
def metrics(actual, forecast, scaler=None, eps=1e-5):
    if scaler is None:
        _actual = actual.values
        _forecast = forecast.values
    else:
        _actual = scaler.inverse_transform(actual.values.T.astype("float64")).T
        _forecast = scaler.inverse_transform(forecast.values.T.astype("float64")).T
    diff = (_actual - _forecast)
    smape = np.mean(2 * np.abs(diff) / (np.abs(_actual) + np.abs(_forecast) + eps), axis=1)
    rmse = np.sqrt(np.mean(diff**2, axis=1))
    res = pd.DataFrame(index=actual.index)
    res["SMAPE"] = smape
    res["RMSE"] = rmse
    return res

## Reading data

In [ ]:
prefix = "gdrive/MyDrive/diploma/data/"

chlorcon = pd.concat([
    pd.read_csv(prefix + "ChlorineConcentration/ChlorineConcentration_TRAIN.tsv", sep="\t", header=None),
    pd.read_csv(prefix + "ChlorineConcentration/ChlorineConcentration_TEST.tsv", sep="\t", header=None)
]).reset_index(drop=True).drop(columns=0)

power_italy = pd.concat([
    pd.read_csv(prefix + "ItalyPowerDemand/ItalyPowerDemand_TRAIN.tsv", sep="\t", header=None),
    pd.read_csv(prefix + "ItalyPowerDemand/ItalyPowerDemand_TEST.tsv", sep="\t", header=None)
]).reset_index(drop=True).drop(columns=0)

power_italy_train, power_italy_test = train_test_split(power_italy, 6)
power_italy_train = TimeSeriesDataset(power_italy_train, horizon=6, context_length=6 * 2)
power_italy_trainloader = torch.utils.data.DataLoader(power_italy_train, batch_size=256,
                                          shuffle=True, num_workers=2)

chlorcon_train, chlorcon_test = train_test_split(chlorcon, 28)
chlorcon_train = TimeSeriesDataset(chlorcon_train, horizon=28, context_length=28 * 3)
chlorcon_trainloader = torch.utils.data.DataLoader(chlorcon_train, batch_size=256,
                                          shuffle=True, num_workers=2)

In [ ]:
prefix = "gdrive/MyDrive/diploma/data/"
m4_hourly = pd.read_csv(prefix + "m4/Hourly.csv").set_index("V1")

m4_hourly_train, m4_hourly_test = train_test_split(m4_hourly, 48)
m4_hourly_train_dataset = TimeSeriesDataset(m4_hourly_train, horizon=48, context_length=48 * 3)
trainloader = torch.utils.data.DataLoader(m4_hourly_train_dataset, batch_size=256,
                                          shuffle=True, num_workers=2)

In [ ]:
prefix = "gdrive/MyDrive/diploma/data/"
calls = pd.read_csv(prefix + "tinkoff/calls.csv").pivot_table(values="cnt", index="direction", columns="date")#.iloc[:, :-1]

calls_train, calls_test = train_test_split(calls, 28)
calls_train_dataset = TimeSeriesDataset(calls_train, horizon=28, context_length=28 * 3)
trainloader = torch.utils.data.DataLoader(calls_train_dataset, batch_size=256,
                                          shuffle=True, num_workers=2)
len(calls_train_dataset)

2584

## MCNN Model [Tinkoff Series]

In [ ]:
EPOCHS = 300
device = torch.device("cuda")

forecaster = Forecaster(
    context=28 * 3,
    horizon=28,
    smooth_degrees=[3, 17, 28],
    ds_rates=[3, 7, 10],
    kernels=112,
    kernel_size=5,
    pooling="avg",
    pooling_factor=4
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=calls_test, 
    df_train=calls_train,
    context_length=28 * 3
)

In [ ]:
calls_forecast = forecast(calls_train, calls_test, forecaster, 28 * 3, device)
calls_metrics = metrics(calls_test, calls_forecast)
calls_metrics.mean()

SMAPE       0.085518
RMSE     1398.638010
dtype: float64

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

In [ ]:
study.best_params

{'kernel_size': 5, 'kernels': 112, 'pooling': 'avg', 'pooling_factor': 4}

## M4 Hourly Series

In [ ]:
scaler = StandardScaler()

In [ ]:
m4_scaled = scaler.fit_transform(m4_hourly.T)
m4_scaled = pd.DataFrame(
    index=m4_hourly.index,
    columns=m4_hourly.columns,
    data=m4_scaled.T
)

In [ ]:
m4_hourly_train, m4_hourly_test = train_test_split(m4_scaled, 48)

m4_hourly_train_dataset = TimeSeriesDataset(m4_hourly_train, horizon=48, context_length=48 * 3)
trainloader = torch.utils.data.DataLoader(m4_hourly_train_dataset, batch_size=256,
                                          shuffle=True, num_workers=2)

### LSTM2LSTM model

In [ ]:
%%time

EPOCHS = 100
device = torch.device("cuda")

forecaster = LSTMForecaster(
    horizon=48,
    hidden=64,
    device=device
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=m4_hourly_test, 
    df_train=m4_hourly_train,
    context_length=48 * 3,
    scaler=scaler
)

In [ ]:
m4_forecast = forecast(m4_hourly_train, m4_hourly_test, forecaster, 48 * 3, device)
m4_metrics = metrics(m4_hourly_test, m4_forecast, scaler=scaler)
m4_metrics.mean()

SMAPE      0.127529
RMSE     416.536122
dtype: float64

### MCNN Model

In [ ]:
def objective(trial):
    device = torch.device("cuda")

    kernels = trial.suggest_int('kernels', 16, 128)
    lstm_hidden = trial.suggest_int('lstm_hidden', 16, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 8)
    pooling_factor = trial.suggest_int('pooling_factor', 2, 5)
    pooling = trial.suggest_categorical("pooling", ["avg", "max"])

    forecaster = ConvLSTM(
        context=48 * 3,
        horizon=48,
        smooth_degrees=[3, 24, 48],
        ds_rates=[3, 7, 10],
        kernels=kernels,
        kernel_size=kernel_size,
        pooling=pooling,
        pooling_factor=pooling_factor,
        lstm_hidden=lstm_hidden
    ).to(device)
    optimizer = optim.Adam(forecaster.parameters())
    loss_function = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)
    train(forecaster, loss_function, optimizer, trainloader, EPOCHS=5, device=device, disp=False, scheduler=scheduler, scaler=scaler)

    df_forecast = forecast(m4_hourly_train, m4_hourly_test, forecaster, 48 * 3, device)
    df_metrics = metrics(m4_hourly_test, df_forecast, scaler=scaler)
    loss = df_metrics.mean()["SMAPE"]

    return loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2021-06-27 13:14:01,053] A new study created in memory with name: no-name-dad0e478-6f0c-4d14-a4ee-6a9e6dcf652c
[I 2021-06-27 13:14:52,764] Trial 0 finished with value: 0.27006428881683225 and parameters: {'kernels': 112, 'kernel_size': 3, 'pooling_factor': 3, 'pooling': 'max'}. Best is trial 0 with value: 0.27006428881683225.


Finished Training


[I 2021-06-27 13:15:43,523] Trial 1 finished with value: 0.16922073108562938 and parameters: {'kernels': 80, 'kernel_size': 4, 'pooling_factor': 3, 'pooling': 'avg'}. Best is trial 1 with value: 0.16922073108562938.


Finished Training


[I 2021-06-27 13:16:33,131] Trial 2 finished with value: 0.17576050505038376 and parameters: {'kernels': 37, 'kernel_size': 5, 'pooling_factor': 5, 'pooling': 'max'}. Best is trial 1 with value: 0.16922073108562938.


...

Finished Training


[I 2021-06-27 13:29:47,239] Trial 18 finished with value: 0.1550106286094574 and parameters: {'kernels': 91, 'kernel_size': 5, 'pooling_factor': 5, 'pooling': 'avg'}. Best is trial 16 with value: 0.1504555265369969.


Finished Training


[I 2021-06-27 13:30:36,233] Trial 19 finished with value: 0.16236745604186711 and parameters: {'kernels': 55, 'kernel_size': 7, 'pooling_factor': 2, 'pooling': 'avg'}. Best is trial 16 with value: 0.1504555265369969.


Finished Training


In [ ]:
study.best_params

{'kernel_size': 7, 'kernels': 29, 'pooling': 'avg', 'pooling_factor': 5}

In [ ]:
%%time

EPOCHS = 100
device = torch.device("cuda")

forecaster = ConvLSTM(
    context=48 * 3,
    horizon=48,
    smooth_degrees=[3, 17, 28],
    ds_rates=[3, 7, 10],
    kernels=29,
    kernel_size=7,
    pooling="avg",
    pooling_factor=5,
    lstm_hidden=90
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=m4_hourly_test, 
    df_train=m4_hourly_train,
    context_length=48 * 3,
    scaler=scaler
)

Epoch 0: Loss=0.2373691923120647, test SMAPE=0.16286330920053874
Epoch 1: Loss=0.2023188362024652, test SMAPE=0.16421410659543476
Epoch 2: Loss=0.19771029606678533, test SMAPE=0.1522938599186389
Epoch 3: Loss=0.19511325310156183, test SMAPE=0.15860138169696908
Epoch 4: Loss=0.1933130639089832, test SMAPE=0.14940194422738332
Epoch 5: Loss=0.19238744544100414, test SMAPE=0.14857423199663386
Epoch 6: Loss=0.19142829811254752, test SMAPE=0.15319487312069463
Epoch 7: Loss=0.19084674128991308, test SMAPE=0.14832857984540557
Epoch 8: Loss=0.19044775991402205, test SMAPE=0.14611871143771707
Epoch 9: Loss=0.1900770475780501, test SMAPE=0.1498879355441721
Epoch 10: Loss=0.18966405484282856, test SMAPE=0.15001598273364963
Epoch 11: Loss=0.1892650539543733, test SMAPE=0.1474763123922478
Epoch 12: Loss=0.18917466081606532, test SMAPE=0.14982631515280767
Epoch 13: Loss=0.18897164947561268, test SMAPE=0.14978381820855668
Epoch 14: Loss=0.18882472599739009, test SMAPE=0.14528458441473405
Epoch 15: Los

In [ ]:
m4_forecast = forecast(m4_hourly_train, m4_hourly_test, forecaster, 48 * 3, device)
m4_metrics = metrics(m4_hourly_test, m4_forecast, scaler=scaler)
m4_metrics.mean()

SMAPE      0.123722
RMSE     414.231976
dtype: float64

## MCNN2LSTM [Tinkoff series]

In [ ]:
prefix = "gdrive/MyDrive/diploma/data/"
calls = pd.read_csv(prefix + "tinkoff/chats2.csv").pivot_table(values="sum", index="product", columns="dt").fillna(method="ffill")
#calls pd.read_csv(prefix + "tinkoff/calls.csv").pivot_table(values="cnt", index="direction", columns="date")
#chats pd.read_csv(prefix + "tinkoff/chats2.csv").pivot_table(values="sum", index="product", columns="dt").fillna(method="ffill")
#sme pd.read_csv(prefix + "tinkoff/sme.csv").pivot_table(values="cnt", index="var", columns="date").iloc[:, :-1]

scaler = StandardScaler()

calls_scaled = scaler.fit_transform(calls.T)
calls_scaled = pd.DataFrame(
    index=calls.index,
    columns=calls.columns,
    data=calls_scaled.T
)

calls_train, calls_test = train_test_split(calls_scaled, 28)
calls_train_dataset = TimeSeriesDataset(calls_train, horizon=28, context_length=28 * 3)

In [ ]:
trainloader = torch.utils.data.DataLoader(calls_train_dataset, batch_size=256,
                                          shuffle=True, num_workers=2)

In [ ]:
def objective(trial):
    device = torch.device("cuda")

    kernels = trial.suggest_int('kernels', 16, 128)
    lstm_hidden = trial.suggest_int('lstm_hidden', 16, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 8)
    pooling_factor = trial.suggest_int('pooling_factor', 2, 5)
    pooling = trial.suggest_categorical("pooling", ["avg", "max"])

    forecaster = ConvLSTM(
        context=28 * 3,
        horizon=28,
        smooth_degrees=[3, 7, 14, 28],
        ds_rates=[3, 7, 10],
        kernels=kernels,
        kernel_size=kernel_size,
        pooling=pooling,
        pooling_factor=pooling_factor,
        lstm_hidden=lstm_hidden
    ).to(device)
    optimizer = optim.Adam(forecaster.parameters())
    loss_function = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)
    train(forecaster, loss_function, optimizer, trainloader, EPOCHS=100, device=device, disp=False, scheduler=scheduler, scaler=scaler)

    df_forecast = forecast(calls_train, calls_test, forecaster, 28 * 3, device)
    df_metrics = metrics(calls_test, df_forecast, scaler=scaler)
    loss = df_metrics.mean()["SMAPE"]

    return loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

[I 2021-06-27 22:34:27,848] A new study created in memory with name: no-name-d76ae496-f5eb-4dd7-9e72-9ec211b5c096
[I 2021-06-27 22:36:10,964] Trial 0 finished with value: 0.4515655048015096 and parameters: {'kernels': 87, 'lstm_hidden': 42, 'kernel_size': 4, 'pooling_factor': 5, 'pooling': 'avg'}. Best is trial 0 with value: 0.4515655048015096.


Finished Training


[I 2021-06-27 22:37:54,393] Trial 1 finished with value: 0.4188082240237375 and parameters: {'kernels': 99, 'lstm_hidden': 73, 'kernel_size': 5, 'pooling_factor': 3, 'pooling': 'max'}. Best is trial 1 with value: 0.4188082240237375.


Finished Training


[I 2021-06-27 22:39:37,626] Trial 2 finished with value: 0.40631371380586095 and parameters: {'kernels': 58, 'lstm_hidden': 39, 'kernel_size': 8, 'pooling_factor': 2, 'pooling': 'avg'}. Best is trial 2 with value: 0.40631371380586095.


...

Finished Training


[I 2021-06-27 23:36:31,171] Trial 28 finished with value: 0.4436211774234173 and parameters: {'kernels': 75, 'lstm_hidden': 26, 'kernel_size': 7, 'pooling_factor': 4, 'pooling': 'avg'}. Best is trial 4 with value: 0.39571099343930516.


Finished Training


[I 2021-06-27 23:38:48,688] Trial 29 finished with value: 0.45289848106830877 and parameters: {'kernels': 90, 'lstm_hidden': 37, 'kernel_size': 4, 'pooling_factor': 5, 'pooling': 'avg'}. Best is trial 4 with value: 0.39571099343930516.


Finished Training


In [ ]:
 study.best_params

{'kernel_size': 3,
 'kernels': 17,
 'lstm_hidden': 109,
 'pooling': 'avg',
 'pooling_factor': 3}

In [ ]:
EPOCHS = 100
device = torch.device("cuda")

forecaster = ConvLSTM(
    context=28 * 3,
    horizon=28,
    smooth_degrees=[3, 7, 14, 28],
    ds_rates=[3, 7, 10],
    kernels=17,
    kernel_size=3,
    pooling="avg",
    pooling_factor=3,
    lstm_hidden=109
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=calls_test, 
    df_train=calls_train,
    context_length=28 * 3,
    scaler=scaler
)

Epoch 0: Loss=0.9706501066684723, test SMAPE=0.4220283995133475
Epoch 1: Loss=0.9581988453865051, test SMAPE=0.41801406328006346
Epoch 2: Loss=0.94124103585879, test SMAPE=0.42015391042049105
Epoch 3: Loss=0.9235957066218058, test SMAPE=0.41369636671105564
Epoch 4: Loss=0.9047057926654816, test SMAPE=0.4118420170730489
Epoch 5: Loss=0.881784717241923, test SMAPE=0.4536740612564008
Epoch 6: Loss=0.8691787719726562, test SMAPE=0.45361512808187376
Epoch 7: Loss=0.8691384891668955, test SMAPE=0.4453758774187539
Epoch 8: Loss=0.8616882562637329, test SMAPE=0.44233514518869266
Epoch 9: Loss=0.8631576796372732, test SMAPE=0.44627742834122197
Epoch 10: Loss=0.8547138571739197, test SMAPE=0.4353100100902926
Epoch 11: Loss=0.8468252519766489, test SMAPE=0.4471497792297112
Epoch 12: Loss=0.8484050631523132, test SMAPE=0.4305270407042561
Epoch 13: Loss=0.841902623573939, test SMAPE=0.4220566783795812
Epoch 14: Loss=0.8161129554112753, test SMAPE=0.4308296479628534
Epoch 15: Loss=0.758487472931544,

In [ ]:
calls_forecast = forecast(calls_train, calls_test, forecaster, 28 * 3, device)
calls_metrics = metrics(calls_test, calls_forecast, scaler=scaler)
calls_metrics.mean()

SMAPE       0.111118
RMSE     1104.541935
dtype: float64

### Mean SMAPE

In [ ]:
(4 * 0.169 + 4 * 0.419 + 2 * 0.449) / 10

0.325

In [ ]:
(4 * 0.095 + 4 * 0.364 + 2 * 0.088) / 10

0.2012

### Chlorine Concentration (LSTM2LSTM)

In [ ]:
EPOCHS = 50
device = torch.device("cuda")

forecaster = LSTMForecaster(
    horizon=28,
    hidden=64,
    device=device
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    chlorcon_trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=chlorcon_test, 
    df_train=chlorcon_train,
    context_length=28 * 3,
    scaler=None
)

Epoch 0: Loss=0.17609192452260425, test SMAPE=1.347066026107311
Epoch 1: Loss=0.05372020806093793, test SMAPE=1.3246597790223744
Epoch 2: Loss=0.03602407488685388, test SMAPE=1.310747602535011
Epoch 3: Loss=0.03015435753942846, test SMAPE=1.2975200268765257
Epoch 4: Loss=0.026024646512590923, test SMAPE=1.312532983598149
Epoch 5: Loss=0.021821193826886322, test SMAPE=1.3327059513051738
Epoch 6: Loss=0.01852467970124313, test SMAPE=1.3434113110133183
Epoch 7: Loss=0.015678928231636247, test SMAPE=1.365739105642741
Epoch 8: Loss=0.013771739606182654, test SMAPE=1.3411591111696632
Epoch 9: Loss=0.012393492853248513, test SMAPE=1.3312989114091374
Epoch 10: Loss=0.011291495135442897, test SMAPE=1.326519867342608
Epoch 11: Loss=0.010505376174882218, test SMAPE=1.3238935254790438
Epoch 12: Loss=0.00965229040318793, test SMAPE=1.332963302578477
Epoch 13: Loss=0.019562079184330426, test SMAPE=1.3276411695050305
Epoch 14: Loss=0.009345965947096165, test SMAPE=1.3348296882245205
Epoch 15: Loss=0.

### Chlorine Concentration (MCNN2LSTM)

In [ ]:
def objective(trial):
    device = torch.device("cuda")

    kernels = trial.suggest_int('kernels', 16, 128)
    lstm_hidden = trial.suggest_int('lstm_hidden', 16, 128)
    kernel_size = trial.suggest_int('kernel_size', 3, 8)
    pooling_factor = trial.suggest_int('pooling_factor', 2, 5)
    pooling = trial.suggest_categorical("pooling", ["avg", "max"])

    forecaster = ConvLSTM(
        context=28 * 3,
        horizon=28,
        smooth_degrees=[3, 7, 28],
        ds_rates=[3, 7, 10],
        kernels=kernels,
        kernel_size=kernel_size,
        pooling=pooling,
        pooling_factor=pooling_factor,
        lstm_hidden=lstm_hidden
    ).to(device)
    optimizer = optim.Adam(forecaster.parameters())
    loss_function = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)
    train(forecaster, loss_function, optimizer, chlorcon_trainloader, EPOCHS=3, device=device, disp=False, scheduler=scheduler)

    df_forecast = forecast(chlorcon_train, chlorcon_test, forecaster, 28 * 3, device)
    df_metrics = metrics(chlorcon_test, df_forecast)
    loss = df_metrics.mean()["SMAPE"]

    return loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

[I 2021-06-28 23:05:32,160] A new study created in memory with name: no-name-536a19a0-8dd7-40dc-a052-96a99408238a
[I 2021-06-28 23:06:24,653] Trial 0 finished with value: 1.3200064334740407 and parameters: {'kernels': 40, 'lstm_hidden': 75, 'kernel_size': 7, 'pooling_factor': 2, 'pooling': 'avg'}. Best is trial 0 with value: 1.3200064334740407.


Finished Training


[I 2021-06-28 23:07:24,016] Trial 1 finished with value: 1.3064188346144874 and parameters: {'kernels': 116, 'lstm_hidden': 39, 'kernel_size': 5, 'pooling_factor': 5, 'pooling': 'avg'}. Best is trial 1 with value: 1.3064188346144874.


Finished Training


[I 2021-06-28 23:08:13,089] Trial 2 finished with value: 1.285596193682334 and parameters: {'kernels': 16, 'lstm_hidden': 18, 'kernel_size': 5, 'pooling_factor': 5, 'pooling': 'avg'}. Best is trial 2 with value: 1.285596193682334.


...

Finished Training


[I 2021-06-28 23:22:48,411] Trial 18 finished with value: 1.322994024217716 and parameters: {'kernels': 112, 'lstm_hidden': 22, 'kernel_size': 4, 'pooling_factor': 2, 'pooling': 'avg'}. Best is trial 6 with value: 1.274043162091727.


Finished Training


[I 2021-06-28 23:23:44,380] Trial 19 finished with value: 1.364301835219351 and parameters: {'kernels': 60, 'lstm_hidden': 52, 'kernel_size': 6, 'pooling_factor': 3, 'pooling': 'max'}. Best is trial 6 with value: 1.274043162091727.


Finished Training


In [ ]:
EPOCHS = 50
device = torch.device("cuda")

forecaster = ConvLSTM(
    context=28 * 3,
    horizon=28,
    smooth_degrees=[3, 7, 28],
    ds_rates=[3, 7, 10],
    kernels=119,
    kernel_size=8,
    pooling="max",
    pooling_factor=3,
    lstm_hidden=24
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    chlorcon_trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=chlorcon_test, 
    df_train=chlorcon_train,
    context_length=28 * 3
)

Epoch 0: Loss=0.15465486519611799, test SMAPE=1.3109241488691579
Epoch 1: Loss=0.056818393085684094, test SMAPE=1.298709899384571
Epoch 2: Loss=0.04240746992629963, test SMAPE=1.3100173539267719
Epoch 3: Loss=0.03434400140502296, test SMAPE=1.330554199401982
Epoch 4: Loss=0.02926782201361525, test SMAPE=1.330422191221757
Epoch 5: Loss=0.026544876957496445, test SMAPE=1.322087890574089
Epoch 6: Loss=0.024212902133445163, test SMAPE=1.328713401989502
Epoch 7: Loss=0.022392956466301458, test SMAPE=1.3393429101308156
Epoch 8: Loss=0.020848290147362173, test SMAPE=1.3468502614833158
Epoch 9: Loss=0.01957002602681354, test SMAPE=1.353599838720269
Epoch 10: Loss=0.01860393586230802, test SMAPE=1.3667935154774606
Epoch 11: Loss=0.017495882773137354, test SMAPE=1.3690952169743758
Epoch 12: Loss=0.016654891553488406, test SMAPE=1.3676031553384966
Epoch 13: Loss=0.01563334717609725, test SMAPE=1.38627990500225
Epoch 14: Loss=0.014645460812927603, test SMAPE=1.374251885072516
Epoch 15: Loss=0.0137

### Italy Power Demand (LSTM2LSTM)

In [ ]:
EPOCHS = 50
device = torch.device("cuda")

forecaster = LSTMForecaster(
    horizon=6,
    hidden=8,
    device=device
).to(device)
optimizer = optim.Adam(forecaster.parameters())
loss_function = nn.MSELoss()
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)


train(
    forecaster, 
    loss_function, 
    optimizer, 
    power_italy_trainloader, 
    EPOCHS=EPOCHS, 
    device=device, 
    disp=True, 
    scheduler=scheduler, 
    df_test=power_italy_test, 
    df_train=power_italy_train,
    context_length=6 * 2,
    scaler=None
)

Epoch 0: Loss=0.637777042388916, test SMAPE=1.6356396768144452
Epoch 1: Loss=0.6130467295646668, test SMAPE=1.6419155325246837
Epoch 2: Loss=0.5878894805908204, test SMAPE=1.649954886726358
Epoch 3: Loss=0.5667909026145935, test SMAPE=1.660845293384791
Epoch 4: Loss=0.5494748711585998, test SMAPE=1.6755753493847843
Epoch 5: Loss=0.5165984451770782, test SMAPE=1.6956451630628686
Epoch 6: Loss=0.49952889084815977, test SMAPE=1.723665350149656
Epoch 7: Loss=0.4696254074573517, test SMAPE=1.7654507749160218
Epoch 8: Loss=0.4534695506095886, test SMAPE=1.833480779573863
Epoch 9: Loss=0.4217736482620239, test SMAPE=1.9048202482830567
Epoch 10: Loss=0.4000580608844757, test SMAPE=1.7669098109424681
Epoch 11: Loss=0.3701344311237335, test SMAPE=1.6397399252333904
Epoch 12: Loss=0.3421397924423218, test SMAPE=1.5276074953528975
Epoch 13: Loss=0.31169477105140686, test SMAPE=1.4314076432584117
Epoch 14: Loss=0.2828073024749756, test SMAPE=1.350061805788182
Epoch 15: Loss=0.2535977602005005, test

### Italy Power Demand (MCNN2LSTM)

In [ ]:
def objective(trial):
    device = torch.device("cuda")

    kernels = trial.suggest_int('kernels', 8, 128)
    lstm_hidden = trial.suggest_int('lstm_hidden', 8, 128)
    kernel_size = trial.suggest_int('kernel_size', 2, 8)
    pooling_factor = trial.suggest_int('pooling_factor', 2, 5)
    pooling = trial.suggest_categorical("pooling", ["avg", "max"])

    forecaster = ConvLSTM(
        context=12,
        horizon=6,
        smooth_degrees=[3, 4, 5],
        ds_rates=[],
        kernels=kernels,
        kernel_size=kernel_size,
        pooling=pooling,
        pooling_factor=pooling_factor,
        lstm_hidden=lstm_hidden
    ).to(device)
    optimizer = optim.Adam(forecaster.parameters())
    loss_function = nn.MSELoss()
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, threshold=1e-4)
    train(forecaster, loss_function, optimizer, power_italy_trainloader, EPOCHS=50, device=device, disp=False, scheduler=scheduler)

    df_forecast = forecast(power_italy_train, power_italy_test, forecaster, 6 * 2, device)
    df_metrics = metrics(power_italy_test, df_forecast)
    loss = df_metrics.mean()["SMAPE"]

    return loss

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

[I 2021-06-29 00:05:27,617] A new study created in memory with name: no-name-8b2564d3-2159-4eaa-b93b-fcac6cd4b39a
[I 2021-06-29 00:05:35,494] Trial 0 finished with value: 1.6541461910511093 and parameters: {'kernels': 53, 'lstm_hidden': 86, 'kernel_size': 5, 'pooling_factor': 4, 'pooling': 'avg'}. Best is trial 0 with value: 1.6541461910511093.


Finished Training


[I 2021-06-29 00:05:43,470] Trial 1 finished with value: 1.737844499976146 and parameters: {'kernels': 51, 'lstm_hidden': 94, 'kernel_size': 3, 'pooling_factor': 3, 'pooling': 'avg'}. Best is trial 0 with value: 1.6541461910511093.


Finished Training


[I 2021-06-29 00:05:51,255] Trial 2 finished with value: 1.6846048807040273 and parameters: {'kernels': 105, 'lstm_hidden': 24, 'kernel_size': 6, 'pooling_factor': 5, 'pooling': 'max'}. Best is trial 0 with value: 1.6541461910511093.


...

Finished Training


[I 2021-06-29 00:20:29,050] Trial 98 finished with value: 1.6708445551363944 and parameters: {'kernels': 120, 'lstm_hidden': 58, 'kernel_size': 8, 'pooling_factor': 3, 'pooling': 'avg'}. Best is trial 92 with value: 1.4945530433321126.


Finished Training


[I 2021-06-29 00:20:39,457] Trial 99 finished with value: 1.6676416380301875 and parameters: {'kernels': 115, 'lstm_hidden': 43, 'kernel_size': 7, 'pooling_factor': 3, 'pooling': 'avg'}. Best is trial 92 with value: 1.4945530433321126.


Finished Training
